#  importing the libraries

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight
from torchvision.datasets import ImageFolder
from timm.models.swin_transformer import swin_base_patch4_window7_224
from sklearn.metrics import balanced_accuracy_score
import numpy as np
from tqdm import tqdm  # for loading bar

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 18


# loading the dataset

In [ ]:

# Data Augmentation
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset
train_dataset = ImageFolder(root='/kaggle/input/misahubdataset/Dataset/training', transform=train_transform)
val_dataset = ImageFolder(root='/kaggle/input/misahubdataset/Dataset/validation', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)




# training the swin model on custom dataset

In [ ]:
# Compute class weights based on training labels
class_weights = compute_class_weight('balanced', classes=np.unique(train_dataset.targets), y=train_dataset.targets)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Define the model
model = swin_base_patch4_window7_224(pretrained=True, num_classes=len(train_dataset.classes))
model = model.to(device)

# Loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

# Track the best model based on balanced accuracy
best_balanced_acc = 0.0
best_model_path = 'swin_transformer_best_model_in_the_world.pth'

# Training loop
for epoch in range(num_epochs):
    print(f"\nEpoch [{epoch+1}/{num_epochs}]")

    # Training Phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    train_progress = tqdm(train_loader, desc="Training", leave=False)
    for images, labels in train_progress:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        train_progress.set_postfix(loss=running_loss / (train_progress.n + 1))

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%")

    # Validation Phase
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    all_labels = []
    all_preds = []

    val_progress = tqdm(val_loader, desc="Validating", leave=False)
    with torch.no_grad():
        for images, labels in val_progress:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

            val_progress.set_postfix(val_loss=running_val_loss / (val_progress.n + 1))

    val_loss = running_val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")
    print(f"Balanced Validation Accuracy: {balanced_acc:.4f}")

    # Check if the current model has the best balanced accuracy
    if balanced_acc > best_balanced_acc:
        best_balanced_acc = balanced_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved with Balanced Accuracy: {best_balanced_acc:.4f}")

print(f"Training complete. Best Balanced Accuracy: {best_balanced_acc:.4f}")

# predicting labels for test dataset

In [2]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset

# Custom Dataset class for your test images
class TestImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith('.jpg')]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_path  # Returning image and image path

# Path to test data directory
test_dir = '/kaggle/input/testdataset-misahub/Testing set/Images'  # Update this to your actual path

# Define test data transformations
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load test dataset using the custom dataset class
test_dataset = TestImageDataset(root_dir=test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Manually define classes to ensure consistency with the training classes
classes = [
    'Angioectasia', 'Bleeding', 'Erosion', 'Erythema', 'Foreign Body',
    'Lymphangiectasia', 'Normal', 'Polyp', 'Ulcer', 'Worms'
]
test_dataset.classes = classes

# Set num_classes to match the trained model's output (in your case it's 10)
num_classes = 10

# Load the saved model and ensure correct number of classes
model = swin_base_patch4_window7_224(pretrained=False, num_classes=num_classes)

# Load the model's weights
# Load the model's weights with map_location to handle CPU-only environments
model.load_state_dict(torch.load('swin_transformer_best_model_in_the_world.pth', map_location=device))
# model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
model.eval()

# Initialize lists to store image paths, class probabilities, and predicted classes
image_paths = []
class_probs = []
predicted_classes = []

# Prediction loop
with torch.no_grad():
    test_progress = tqdm(test_loader, desc="Generating Predictions")
    for images, img_paths in test_progress:  # Now, using img_paths returned by the dataset
        images = images.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        
        # Forward pass
        outputs = model(images)
        
        # Apply softmax to get probabilities
        probs = torch.softmax(outputs, dim=1)
        
        # Get predicted class (the one with the highest probability)
        _, predicted = torch.max(probs, 1)
        
        # Store probabilities and predicted class
        class_probs.extend(probs.cpu().numpy())
        predicted_classes.extend(predicted.cpu().numpy())
        image_paths.extend(img_paths)  # Directly extend the image paths

# Convert predicted class indices to class names safely
predicted_labels = []
for pred in predicted_classes:
    if pred < len(test_dataset.classes):
        predicted_labels.append(test_dataset.classes[pred])
    else:
        predicted_labels.append('Unknown')  # Handle any out-of-range predictions

# Create a DataFrame with class probabilities and predicted labels
columns = ['image_path'] + test_dataset.classes + ['predicted_class']
data = []

for i, image_path in enumerate(image_paths):
    row = [image_path] + class_probs[i].tolist() + [predicted_labels[i]]
    data.append(row)

output_df = pd.DataFrame(data, columns=columns)

# Save the DataFrame to an Excel file (.xlsx)
output_df.to_excel('test_predictions_with_probs.xlsx', index=False)

print("Predictions saved to 'test_predictions_with_probs.xlsx'")



# evaluating the model

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from timm.models.swin_transformer import swin_base_patch4_window7_224
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Augmentation for validation
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load validation dataset
val_dataset = ImageFolder(root='/kaggle/input/misahubdataset/Dataset/validation', transform=val_transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load the model (assuming it was saved with 10 classes)
num_classes = 10
model = swin_base_patch4_window7_224(pretrained=False, num_classes=num_classes)
model = model.to(device)

# Load the trained model weights
model.load_state_dict(torch.load('swin_transformer_best_model_in_the_world.pth', map_location=device))

# Set the model to evaluation mode
model.eval()

# Define loss function (if you want to report validation loss, otherwise you can skip this)
criterion = nn.CrossEntropyLoss()

# Initialize metrics storage
all_labels = []
all_preds = []
running_val_loss = 0.0
correct_val = 0
total_val = 0

# Evaluation loop over the validation dataset
with torch.no_grad():
    val_progress = tqdm(val_loader, desc="Validating")
    for images, labels in val_progress:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

        # Calculate loss (optional)
        loss = criterion(outputs, labels)
        running_val_loss += loss.item()

        # Get predictions
        _, predicted = torch.max(outputs.data, 1)

        # Store all true and predicted labels
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

        # Calculate correct predictions for accuracy
        total_val += labels.size(0)
        correct_val += (predicted == labels).sum().item()

# Calculate validation loss
val_loss = running_val_loss / len(val_loader)

# Calculate accuracy
val_accuracy = 100 * correct_val / total_val
balanced_acc = balanced_accuracy_score(all_labels, all_preds)

# Print basic accuracy metrics
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.2f}%")
print(f"Balanced Validation Accuracy: {balanced_acc:.4f}")

# Generate classification report
class_report = classification_report(all_labels, all_preds, target_names=val_dataset.classes, digits=4)
print("\nClassification Report:")
print(class_report)

# Calculate and print other metrics
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')
overall_acc = accuracy_score(all_labels, all_preds)

print(f"Overall Accuracy: {overall_acc:.4f}")
print(f"Precision (weighted): {precision:.4f}")
print(f"Recall (weighted): {recall:.4f}")
print(f"F1 Score (weighted): {f1:.4f}")

# Generate confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds)
print("\nConfusion Matrix:")
print(conf_matrix)


Validation Loss: 0.7446, Validation Accuracy: 75.00%
Balanced Validation Accuracy: 0.6817
Classification Report:
                  precision    recall  f1-score   support

    Angioectasia     0.3142    0.7606    0.4447       497
        Bleeding     0.6063    0.6435    0.6243       359
         Erosion     0.3276    0.5584    0.4129      1155
        Erythema     0.1460    0.5387    0.2297       297
    Foreign Body     0.9643    0.7147    0.8209       340
Lymphangiectasia     0.7125    0.8455    0.7733       343
          Normal     0.9899    0.7825    0.8741     12287
           Polyp     0.4718    0.4860    0.4788       500
           Ulcer     0.4592    0.8846    0.6045       286
           Worms     0.9111    0.6029    0.7257        68

        accuracy                         0.7500     16132
       macro avg     0.5903    0.6817    0.5989     16132
    weighted avg     0.8654    0.7500    0.7895     16132

Overall Accuracy: 0.7500
Precision (weighted): 0.8654
Recall (weighted):